# Redes Adversárias Generativas Condicionais (cGANs)

Neste notebook, exploraremos uma extensão poderosa das Redes Adversárias Generativas (GANs) padrão: as GANs Condicionais, ou cGANs. Enquanto uma GAN tradicional aprende a mapear um vetor de ruído latente $z$ para uma amostra de dados (por exemplo, uma imagem), sem controle sobre qual amostra é gerada, uma cGAN introduz informação adicional $y$ (como um rótulo de classe) tanto no Gerador quanto no Discriminador. Isso nos permite controlar explicitamente as características das amostras geradas.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

## Preparação dos Dados

Agora, carregamos o dataset MNIST. Para agilizar o processo de treinamento, utilizaremos um subconjunto reduzido. O `ToTensor()` transforma as imagens do formato PIL para tensores e normaliza os pixels para o intervalo $[0, 1]$.

In [ ]:
batch_size = 128
img_size = 28 * 28
num_classes = 10

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # Normaliza para [-1, 1]
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

batch_size = 128
img_size = train_dataset[0][0].shape[1] * train_dataset[0][0].shape[2]
num_classes = len(set(train_dataset.targets.numpy()))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Número de batches de treino: {len(train_dataloader)}")
print(f"Número de batches de teste: {len(test_dataloader)}")

## Teoria das cGANs

Uma GAN padrão consiste em duas redes neurais, o Gerador ($G$) e o Discriminador ($D$), que competem em um jogo de soma zero. O Gerador tenta criar dados sintéticos que se assemelham aos dados reais, enquanto o Discriminador tenta distinguir entre amostras reais e falsas. A função de valor (objetivo) para este jogo é:

$$
\min_{G} \max_{D} V(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_{z}(z)}[\log(1 - D(G(z)))]
$$

Nas cGANs, estendemos este framework ao condicionar ambas as redes a uma informação extra $y$. Essa informação pode ser um rótulo de classe, parte de uma imagem ou dados de outra modalidade. O vetor de ruído latente $z$ e a condição $y$ são combinados para alimentar o Gerador. O Discriminador, por sua vez, recebe como entrada tanto a imagem (real ou falsa) quanto a condição $y$.

A nova função objetivo se torna:

$$
\min_{G} \max_{D} V(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x|y)] + \mathbb{E}_{z \sim p_{z}(z)}[\log(1 - D(G(z|y)))]
$$

Dessa forma, o Gerador aprende a gerar amostras que não apenas parecem reais, mas que também correspondem à condição $y$ fornecida.

## Implementação dos Modelos

Para implementar a condicionalidade, usaremos uma camada de `Embedding` para transformar os rótulos de classe (números inteiros de 0 a 9) em vetores densos. Esses vetores serão concatenados tanto com o ruído latente na entrada do Gerador quanto com a imagem na entrada do Discriminador.

### Gerador

O Gerador receberá um vetor de ruído latente e um rótulo de classe. Ele combinará essas duas informações e produzirá uma imagem com as dimensões do MNIST.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, num_classes, img_size):
        super().__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, img_size),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_embedding = self.label_embedding(labels)
        gen_input = torch.cat((noise, label_embedding), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), 1, 28, 28)
        return img

### Discriminador

O Discriminador receberá uma imagem e um rótulo de classe. Seu objetivo é determinar se a imagem é uma amostra real correspondente àquele rótulo.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_classes, img_size):
        super().__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(num_classes + img_size, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        img_flat = img.view(img.size(0), -1)
        label_embedding = self.label_embedding(labels)
        d_in = torch.cat((img_flat, label_embedding), -1)
        validity = self.model(d_in)
        return validity

In [ ]:
latent_dim = 100
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_epochs = 50

generator = Generator(latent_dim, num_classes, img_size).to(device)
discriminator = Discriminator(num_classes, img_size).to(device)

In [ ]:
# Função de custo
adversarial_loss = torch.nn.BCELoss()

# Otimizadores
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

## Loop de Treinamento

O treinamento de uma GAN envolve um processo de duas etapas dentro de cada época:

1.  **Treinamento do Discriminador**:
    * Apresentamos ao Discriminador um lote de imagens reais com seus rótulos correspondentes e calculamos sua perda. O objetivo é que ele classifique essas imagens como `1` (real).
    * Geramos um lote de imagens falsas usando o Gerador, com rótulos aleatórios. Apresentamos essas imagens falsas ao Discriminador e calculamos sua perda. O objetivo é que ele classifique essas como `0` (falso).
    * A perda total do Discriminador é a soma das perdas nas amostras reais e falsas. Atualizamos os pesos do Discriminador através de retropropagação.

2.  **Treinamento do Gerador**:
    * Geramos um novo lote de imagens falsas.
    * Passamos essas imagens pelo Discriminador.
    * Calculamos a perda do Gerador com base na saída do Discriminador, mas desta vez, o objetivo do Gerador é "enganar" o Discriminador. Portanto, a perda é calculada comparando a saída do Discriminador com rótulos de `1` (real).
    * Atualizamos os pesos do Gerador.

In [ ]:
num_epochs = 10
plot_interval = 1
g_losses, d_losses = [], []

for epoch in range(num_epochs):
    generator.train()
    discriminator.train()
    g_loss, d_loss = 0, 0

    for imgs, labels in train_dataloader:
        imgs, labels = imgs.to(device), labels.to(device)
        N = imgs.size(0)

        valid = torch.ones(N, 1, device=device)
        fake = torch.zeros(N, 1, device=device)

        # --- Discriminador ---
        optimizer_D.zero_grad()
        real_pred = discriminator(imgs, labels)
        loss_d_real = adversarial_loss(real_pred, valid)

        z = torch.randn(N, latent_dim, device=device)
        gen_labels = torch.randint(0, num_classes, (N,), device=device)
        fake_imgs = generator(z, gen_labels).detach()

        fake_pred = discriminator(fake_imgs, gen_labels)
        loss_d_fake = adversarial_loss(fake_pred, fake)

        loss_d = 0.5 * (loss_d_real + loss_d_fake)
        loss_d.backward()
        optimizer_D.step()

        # --- Gerador ---
        optimizer_G.zero_grad()
        z = torch.randn(N, latent_dim, device=device)
        gen_labels = torch.randint(0, num_classes, (N,), device=device)
        gen_imgs = generator(z, gen_labels)

        validity = discriminator(gen_imgs, gen_labels)
        loss_g = adversarial_loss(validity, valid)
        loss_g.backward()
        optimizer_G.step()

        d_loss += loss_d.item()
        g_loss += loss_g.item()

    g_losses.append(g_loss / len(train_dataloader))
    d_losses.append(d_loss / len(train_dataloader))

    if epoch % plot_interval == 0 or epoch == num_epochs - 1:
        print(f"[{epoch}/{num_epochs-1}] D: {d_losses[-1]:.4f} | G: {g_losses[-1]:.4f}")
        generator.eval()
        with torch.no_grad():
            z = torch.randn(num_classes, latent_dim, device=device)
            sample_labels = torch.arange(0, num_classes, device=device) % num_classes
            gen_imgs = generator(z, sample_labels).cpu()

        fig, axs = plt.subplots(2, num_classes // 2, figsize=(num_classes, 4))
        for i, ax in enumerate(axs.flat):
            ax.imshow(gen_imgs[i].squeeze(), cmap="gray")
            ax.axis("off")
        plt.suptitle(f"Época {epoch}")
        plt.show()

### Análise dos Resultados

Após o treinamento, podemos analisar os resultados de duas formas principais: visualizando a curva de perda do Gerador e do Discriminador e observando as imagens geradas ao longo do tempo. Idealmente, as perdas de $D$ e $G$ devem convergir para um estado de equilíbrio, embora na prática elas flutuem bastante. A análise mais importante é a qualidade visual das imagens geradas.

In [ ]:
# Plotar o gráfico de perdas
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses, label="G")
plt.plot(d_losses, label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Análise dos Resultados

Durante o treinamento, é esperado que as perdas do Gerador e do Discriminador flutuem. Idealmente, elas devem atingir um equilíbrio, onde nenhuma das redes supera completamente a outra. Se a perda do Discriminador cair para perto de zero, significa que ele está identificando as imagens falsas com muita facilidade, e o Gerador não está aprendendo. Se a perda do Gerador cair muito, o Discriminador pode não estar aprendendo a distinguir as amostras. O equilíbrio de Nash é o ponto ótimo, mas é notoriamente difícil de alcançar na prática.

In [ ]:
def generate_digit(digit, num_samples=10):
    # Preparar ruído e rótulos
    z = torch.randn(num_samples, latent_dim, device=device)
    labels = torch.full((num_samples,), digit, dtype=torch.long, device=device)
    
    # Gerar imagens
    generator.eval() # Modo de avaliação
    with torch.no_grad():
        generated_imgs = generator(z, labels)
    generator.train() # Voltar para o modo de treino
    
    # Desnormalizar e exibir
    generated_imgs = 0.5 * generated_imgs + 0.5 # Desfaz a normalização [-1, 1] para [0, 1]
    
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 2))
    fig.suptitle(f'Dígito {digit}', fontsize=16)
    for i in range(num_samples):
        axs[i].imshow(generated_imgs[i, 0].cpu().numpy(), cmap='gray')
        axs[i].axis('off')
    plt.show()

# Gerar exemplos para cada dígito
for i in range(10):
    generate_digit(i, num_samples=10)

## Exercícios

### Exercício 1

Treine uma cGAN em que as labels sejam 1 ou 0 para números ímpares ou pares no MNIST.